# Molecular Dynamics
## Initializing a system to a specific density

<a id="toc"></a>
## Table of Contents
- [initialize a system](#init)

In [ ]:
import foyer
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np
import parmed as pmd

<a id="init"></a>
## Starting Structure


[back to top](#toc)

In [2]:
propane = mb.load("CCC", smiles=True)

propane.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In the above visualization, carbon atoms are colored grey and hydrogens are white. 

Let's say we want a system of liquid propane at a density of 0.6 g/mL.
mBuild and foyer use these [units](http://docs.openmm.org/7.2.0/userguide/theory.html#units) (nm, amu)

1 amu = 1.66054e-24 g

1 mL = 1 cm$^{3}$ = 1e21 nm$^{3}$

For simplicity, let's say that each propane molecule is 44 amu (3 carbons @12 amu + and 8 hydrogens @1 amu)

If our box volume is 2x2x2 nm (8 nm$^{3}$), then how many propane molecules do we need to fill it at this density?

In [3]:
target_density = 0.6 /1e21 / 1.66054e-24
print(f"The target density is {target_density:.2f} in amu/nm^3")

number = int(round(target_density * 8 / 44))
print(f"This means we need {number} propane molecules to fill a 2x2x2 nm box")

The target density is 361.33 in amu/nm^3
This means we need 66 propane molecules to fill a 2x2x2 nm box


In [4]:
box = mb.Box([2,2,2])
system_box = mb.packing.fill_box(propane, number, box=box)
system_box.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<a id="ff"></a>
## Apply forcefield


_Side note: The `foyer.Forcefield.apply` function currently generates a [parmed structure](https://parmed.github.io/ParmEd/html/structure.html), which does not have the same convenient system-building operations as mbuild. This is why we can't fill the box with our typed molecule. The typed molecule was shown above simply to demonstrate which atoms are assigned the different types. With the new [GMSO](https://gmso.mosdef.org/en/latest/) back-end, this workflow may change._

[back to top](#toc)

In [5]:
system_box_pmd = system_box.to_parmed(box=box)
oplsaa = foyer.forcefields.load_OPLSAA()
box_struc = oplsaa.apply(system_box_pmd, assert_dihedral_params=False)

/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 792, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(ms

<a id="run"></a>
## Run simulation

Now that we have our simulation volume, we can set up our MD simulation to run in [HOOMD](https://hoomd-blue.readthedocs.io/en/stable/) using mbuild's `create_hoomd_simulation` funciton. This will read in the atom positions, bonding information, and forcefield parameters that we have already set. Additionally we need to tell HOOMD about the temperature ([kT, really the thermal energy](https://hoomd-blue.readthedocs.io/en/stable/units.html)), the thermostat coupling (tau), the cutoff where the particles no longer "feel" each other (r_cut), and the time step (dt). Everything else set in the cell below is telling HOOMD how often/when to write out data. The [gsd](https://gsd.readthedocs.io/en/stable/) files will contain the simulation snapshots and the log file will record quantities as the simulation progresses. 

[back to top](#toc)

In [6]:
create_hoomd_simulation(box_struc, r_cut=1.2, auto_scale=True)

HOOMD-blue 2.9.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/18/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Lui, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


*Warning*: pos 15:-1.42643201351 -2.85887551308 -0.272718280554
*Warning*: fractional pos :0.250374397635 -0.000303214788437 0.452274300903
*Warning*: lo: -2.85714285714 -2.85714285714 -2.85714285714
*Warning*: hi: 2.85714285714 2.85714285714 2.85714285714
*Warning*: Not all particles were found inside the given box


RuntimeError: Error initializing ParticleData

Talk about error

In [4]:
box = mb.Box([10,10,10])
system_box = mb.packing.fill_box(propane, number, box=box)
system_box.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
system_box_pmd = system_box.to_parmed(box=box)
oplsaa = foyer.forcefields.load_OPLSAA()
box_struc = oplsaa.apply(system_box_pmd, assert_dihedral_params=False)

/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/Users/jenny/miniconda3/envs/cmelab/lib/python3.7/site-packages/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 792, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(ms

In [6]:
create_hoomd_simulation(box_struc, r_cut=1.2, auto_scale=False);

HOOMD-blue 2.9.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/18/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Lui, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 726 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 4 exclusions             : 528
notice(2): Particles with 7 exclusions             : 132
notice(2): Particles with 10 exclusions             : 66
notice(2): Neighbors included by diameter          : n

In [7]:
# remove pppm force?
forces = [
    f for f in hoomd.context.current.forces 
    if not isinstance(f, hoomd.md.charge.pppm)
]
hoomd.context.current.forces = forces
hoomd.context.current.forces

In [8]:
_all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0001)
integrator = hoomd.md.integrate.nvt(group=_all, kT=1.0, tau=1)
hoomd.dump.gsd("start.gsd", period=None, group=_all, overwrite=True)
hoomd.dump.gsd("traj.gsd", period=1e5, group=_all, phase=0, overwrite=True);

In [9]:
hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,10), (1e4,10), (1e6,2)]));

In [10]:
hoomd.run(1e6)
hoomd.dump.gsd("out.gsd", period=None, group=_all, overwrite=True);

** starting run **


**ERROR**: Particle with unique tag 558 is no longer in the simulation box.

Cartesian coordinates: 
x: -55563.7 y: -358620 z: -67557.3
Fractional coordinates: 
f.x: -5555.87 f.y: -35861.5 f.z: -6755.23
Local box lo: (-5, -5, -5)
          hi: (5, 5, 5)


RuntimeError: Error computing cell list